In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
if (code_show){
$('div.input').hide();
} else {
$('div.input').show();
}
code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [12]:
import glob
import os

In [3]:
import numpy as np
import pandas as pd

In [4]:
import ipywidgets

In [5]:
import bokeh.plotting
import bokeh.layouts
import bokeh.models
import bokeh.io
from bokeh.palettes import Category10_10 as palette

In [6]:
bokeh.plotting.output_notebook()

Loading BokehJS ...

In [7]:
import logging
logging.basicConfig(format='%(levelname)s: %(message)s', level=logging.DEBUG)

# Housing Affordability Data System
Data provided by:<br />
Office of Policy Development and Research (PD&R)<br />
U.S. Department of Housing and Urban Development<br />
Secretary Ben Carson<br />
https://www.huduser.gov/portal/datasets/hads/hads.html

## Load the data

In [8]:
! ls -lh data/

total 651M
-rw-rw-r-- 1 schowell schowell  37M Feb 28  2007 hads1985.txt
-rw-rw-r-- 1 schowell schowell  37M Feb 28  2007 hads1987.txt
-rw-rw-r-- 1 schowell schowell  39M Feb 28  2007 hads1989.txt
-rw-rw-r-- 1 schowell schowell  38M Feb 28  2007 hads1991.txt
-rw-rw-r-- 1 schowell schowell  41M Oct  3  2007 hads1993.txt
-rw-rw-r-- 1 schowell schowell  38M Feb 28  2007 hads1995.txt
-rw-rw-r-- 1 schowell schowell  36M Feb 28  2007 hads1997.txt
-rw-rw-r-- 1 schowell schowell  40M Feb 28  2007 hads1999.txt
-rw-rw-r-- 1 schowell schowell  36M Feb 28  2007 hads2001.txt
-rw-rw-r-- 1 schowell schowell  41M Feb 28  2007 hads2003.txt
-rw-rw-r-- 1 schowell schowell  37M Jan 19  2011 hads2005.txt
-rw-rw-r-- 1 schowell schowell  34M Jan 19  2011 hads2007.txt
-rw-rw-r-- 1 schowell schowell  39M Jun 25  2010 hads2009.txt
-rw-r----- 1 schowell schowell 115M Apr 12  2013 hads2011.txt
-rw-r----- 1 schowell schowell  51M Jun 24  2015 hads2013.txt


The entire data set is 651M, let only grab some of these files.

In [9]:
fnames = glob.glob('data/*txt')
fnames.sort()

In [10]:
fnames

['data/hads1985.txt',
 'data/hads1987.txt',
 'data/hads1989.txt',
 'data/hads1991.txt',
 'data/hads1993.txt',
 'data/hads1995.txt',
 'data/hads1997.txt',
 'data/hads1999.txt',
 'data/hads2001.txt',
 'data/hads2003.txt',
 'data/hads2005.txt',
 'data/hads2007.txt',
 'data/hads2009.txt',
 'data/hads2011.txt',
 'data/hads2013.txt']

In [13]:
years = np.array([int(fname.strip('data/hads.txt')) for fname in fnames])

# print(years)  # too much data to start with 
print(years[::3])  # lets some years

data = {}
columns = []
for year in years[::3]:
    fname = 'data/hads{}.txt'.format(year)
    assert os.path.exists, 'check input, no such file path/name: {}'.format(fname)
    data[year] = pd.read_csv(fname, skiprows=0, sep=',')
    data[year].columns = map(str.lower, data[year].columns)  # make column labels lowercase
    data[year]['year'] = year  # add the year as a feature
    columns.append(data[year].columns)
    

years = years[::3]

[1985 1991 1997 2003 2009]


In [14]:
columns[0][1] in columns[-1]

True

In [15]:
# how similar are the columns?
for i, i_columns in enumerate(columns):
    for column in i_columns:
        if column not in columns[-1]:
            print('`{}` values, from {} data, is not in the 2009 data'.format(column, years[i]))

`metro` values, from 1985 data, is not in the 2009 data
`istatus` values, from 1985 data, is not in the 2009 data
`age` values, from 1985 data, is not in the 2009 data
`fmtmetro` values, from 1985 data, is not in the 2009 data
`metro` values, from 1991 data, is not in the 2009 data
`istatus` values, from 1991 data, is not in the 2009 data
`age` values, from 1991 data, is not in the 2009 data
`fmtmetro` values, from 1991 data, is not in the 2009 data


Identified that the columns did not have the same case.  After reformatting the all to lowercase, there are only a few missing, `metro`, `istatus`, `age`, and `fmtmetro`.  Lets combine these into a single DataFrame.

In [16]:
df_raw = pd.concat([data[key] for key in data.keys()])

In [17]:
df_raw.head()

,abl30,abl50,abl80,ablmed,age,age1,aplmed,assisted,bedrms,built,...,type,utility,vacancy,value,vchrmov,weight,year,zadeq,zinc2,zsmhc
0,NaN,11957.724480,19127.189120,23424.29440,39.0,NaN,22523.3600,0,3,81,...,1,40.000000,-9,-9,NaN,2883.27,1985,1,18000,476
1,NaN,12700.460000,20322.648000,25404.17040,40.0,NaN,25404.1704,0,2,81,...,1,8.000000,-9,-9,NaN,2785.89,1985,1,14200,383
2,NaN,9346.323830,14956.772553,18249.08400,-9.0,NaN,-9.0000,0,2,84,...,1,0.000000,1,-9,NaN,2450.53,1985,1,-9,425
3,NaN,10346.310000,16550.869500,20271.02400,19.0,NaN,18018.6880,0,2,85,...,1,15.750000,-9,-9,NaN,2306.52,1985,1,14000,371
4,NaN,17066.681956,25737.559467,33612.27168,28.0,NaN,29087.5428,0,3,84,...,1,34.166667,-9,160000,NaN,2504.35,1985,1,35000,809


For now lets drop columns that contain `NaN` values.

In [18]:
df = df_raw.dropna(axis=1, how='any')

In [19]:
df.head()

,abl50,abl80,ablmed,aplmed,assisted,bedrms,built,burden,control,cost06,...,totsal,type,utility,vacancy,value,weight,year,zadeq,zinc2,zsmhc
0,11957.724480,19127.189120,23424.29440,22523.3600,0,3,81,0.317333,'100006110249',476.000000,...,18000,1,40.000000,-9,-9,2883.27,1985,1,18000,476
1,12700.460000,20322.648000,25404.17040,25404.1704,0,2,81,0.323662,'100007130148',383.000000,...,13000,1,8.000000,-9,-9,2785.89,1985,1,14200,383
2,9346.323830,14956.772553,18249.08400,-9.0000,0,2,84,-9.000000,'100008700141',425.000000,...,-9,1,0.000000,1,-9,2450.53,1985,1,-9,425
3,10346.310000,16550.869500,20271.02400,18018.6880,0,2,85,0.318000,'100014110140',371.000000,...,14000,1,15.750000,-9,-9,2306.52,1985,1,14000,371
4,17066.681956,25737.559467,33612.27168,29087.5428,0,3,84,0.277371,'100014350142',1122.519423,...,35000,1,34.166667,-9,160000,2504.35,1985,1,35000,809


### What is the distribution of different features and how do these change over time?
Lets explore the data visually.

In [20]:
int(np.where(years == 1997)[0])

2

In [21]:
colors = df.year.apply(lambda year: palette[int(np.where(years == year)[0])])

df = df.assign(color=pd.Series(colors, index=df.index))

Lets start by just looking at one years worth of data.

In [22]:
year = 1985

In [23]:
def update(x='bedrms', y='cost06'):
    r.data_source.data['x'] = df[df.year == year][x]
    r.data_source.data['y'] = df[df.year == year][y]
    p.xaxis.axis_label = x
    p.yaxis.axis_label = y
    bokeh.io.push_notebook()

In [24]:
p = bokeh.plotting.figure(x_axis_label='bedrms', y_axis_label='cost06')
x = df[df.year == year].bedrms
y = df[df.year == year].cost06
colors = df[df.year == year].color
r = p.circle(x, y, color=colors, alpha=0.2)
bokeh.plotting.show(p, notebook_handle=True)

In [25]:
ipywidgets.interact(update, x=list(df.columns), y=list(df.columns))

<function __main__.update>

There are obviously some strange values used for missing data. To handle this appropriately, we need to better understand how the data was measured.

Maybe it will be more revealing to plot use a boxplot: http://bokeh.pydata.org/en/latest/docs/gallery/boxplot.html

In [26]:
# find the quartile and IQR for each category by year
df_by_year = df.groupby('year')

q1 = df_by_year.quantile(q=0.25)
q2 = df_by_year.quantile(q=0.5)
q3 = df_by_year.quantile(q=0.75)
iqr = q3 - q1
upper = q3 + 1.5 * iqr
upper = q1 - 1.5 * iqr

# get the range for the stems
qmin = df_by_year.quantile(q=0.0)
qmax = df_by_year.quantile(q=1.0)

In [27]:
# make string version for x-labels
yrs_str = [str(year) for year in years]

In [28]:
def update_box(feature='cost06'):
    # update stems
    s1.data_source.data['y0'] = qmin[feature]
    s1.data_source.data['y1'] = q1[feature]
    s2.data_source.data['y0'] = q3[feature]
    s2.data_source.data['y1'] = qmax[feature]
    
    # update boxes
    v1.data_source.data['bottom'] = q1[feature]
    v1.data_source.data['top'] = q2[feature]
    v2.data_source.data['bottom'] = q2[feature]
    v2.data_source.data['top'] = q3[feature]

    # update whiskers
    r1.data_source.data['y'] = qmin[feature]
    r2.data_source.data['y'] = qmax[feature]

    p.yaxis.axis_label = feature
    bokeh.io.push_notebook()

In [29]:
initial = 'cost06'
p = bokeh.plotting.figure(x_range=yrs_str, y_axis_label=initial)

# stems
s1 = p.segment(yrs_str, qmin[initial], yrs_str, q1[initial], line_color='black')
s2 = p.segment(yrs_str, q3[initial], yrs_str, qmax[initial], line_color='black')

# boxes
b_width = 0.7
v1 = p.vbar(yrs_str, b_width, q2[initial], q1[initial], fill_color=palette[1], line_color='black')
v2 = p.vbar(yrs_str, b_width, q3[initial], q2[initial], fill_color=palette[0], line_color='black')

# whiskers
w_width = 0.2
r1 = p.rect(yrs_str, qmin[initial], w_width, 0.001, fill_color='black', line_color='black')
r2 = p.rect(yrs_str, qmax[initial], b_width, 0.001, fill_color='black', line_color='black')

p.xgrid.grid_line_color = None

p.xaxis.major_label_text_font_size = "12pt"

bokeh.plotting.show(p, notebook_handle=True)

In [27]:
ipywidgets.interact(update_box, feature=list(df.columns))

<function __main__.update_box>